In [56]:
!pip install segment-anything

In [57]:
import torch
import pandas as pd
import numpy as np
import cv2
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import ViTForImageClassification, ViTFeatureExtractor
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
from google.colab import drive
import os
from sklearn.model_selection import train_test_split

In [58]:
drive.mount('/content/drive')
metadata_path = '/content/drive/MyDrive/Курсовая/HAM10000_metadata.tab'
image_folder_1 = '/content/drive/MyDrive/Курсовая/HAM10000_images_part_1/'
image_folder_2 = '/content/drive/MyDrive/Курсовая/HAM10000_images_part_2/'
processed_folder = '/content/drive/MyDrive//Курсовая/HAM10000_preprocessed/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
df = pd.read_csv(metadata_path, sep='\t', header=None)
df.columns = ['lesion_id', 'image_id', 'diagnosis', 'type_of_diagnostic', 'age', 'sex', 'localization', 'source']
df

,lesion_id,image_id,diagnosis,type_of_diagnostic,age,sex,localization,source
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern
...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,vidir_modern
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,vidir_modern
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,vidir_modern
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,vidir_modern


In [60]:
df.nunique()

,0
lesion_id,7470
image_id,10015
diagnosis,7
type_of_diagnostic,4
age,18
sex,3
localization,15
source,4


In [61]:
df['diagnosis'].unique()

array(['bkl', 'nv', 'df', 'mel', 'vasc', 'bcc', 'akiec'], dtype=object)

bkl — Benign keratosis-like lesions (доброкачественный кератоз)

nv — Melanocytic nevus (меланоцитарный невус)

df — Dermatofibroma (дерматофиброма)

mel — Melanoma (меланома)

vasc — Vascular lesions (сосудистые поражения)

bcc — Basal cell carcinoma (базально-клеточная карцинома)

akiec — Actinic keratosis and intraepithelial carcinoma (актинический кератоз и внутрикожная карцинома)


In [62]:
df['image_path'] = df['image_id'].apply(lambda x: image_folder_1 + x + '.jpg' if os.path.exists(image_folder_1 + x + '.jpg') else image_folder_2 + x + '.jpg')

In [63]:
df

,lesion_id,image_id,diagnosis,type_of_diagnostic,age,sex,localization,source,image_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...
...,...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...


In [64]:
# закодируем диагнозы цифрами
class_names = df['diagnosis'].unique()
class_to_idx = {class_name: idx for idx, class_name in enumerate(class_names)}
df['label_diagnosis'] = df['diagnosis'].map(class_to_idx)
df

,lesion_id,image_id,diagnosis,type_of_diagnostic,age,sex,localization,source,image_path,label_diagnosis
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...,0
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...,0
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...,0
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...,0
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...,0
...,...,...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...,6
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...,6
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...,6
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,vidir_modern,/content/drive/MyDrive/Курсовая/HAM10000_image...,6


In [65]:
df.drop(columns=['diagnosis'], inplace=True)

In [66]:
class_to_idx  # соответствие диагноза лейблу

{'bkl': 0, 'nv': 1, 'df': 2, 'mel': 3, 'vasc': 4, 'bcc': 5, 'akiec': 6}

In [67]:
train_df, tmp_df = train_test_split(df, test_size=0.3, stratify=df['label_diagnosis'], random_state=42)
val_df, test_df = train_test_split(tmp_df, test_size=0.5, stratify=tmp_df['label_diagnosis'], random_state=42)

In [68]:
print(len(train_df), len(val_df), len(test_df))

7010 1502 1503


In [69]:
def segment_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    masks = mask_generator.generate(image)
    return masks

In [70]:
# уберем пока это
'''def preprocess_and_save_images(dataframe):
    os.makedirs(processed_folder, exist_ok=True)
    for idx, row in dataframe.iterrows():
        img_path = image_folder_1 + row['image_id'] + '.jpg' if os.path.exists(image_folder_1 + row['image_id'] + '.jpg') else image_folder_2 + row['image_id'] + '.jpg'
        masks = segment_image(img_path)
        image = cv2.imread(img_path)
        for i, mask in enumerate(masks):
            mask_uint8 = (mask['segmentation'] * 255).astype(np.uint8)
            segmented_image = cv2.bitwise_and(image, image, mask=mask_uint8)
            processed_path = os.path.join(processed_folder, f"{row['image_id']}_mask_{i}.jpg")
            cv2.imwrite(processed_path, segmented_image)

        dataframe.at[idx, 'image_path'] = processed_path


preprocess_and_save_images(df)'''

'def preprocess_and_save_images(dataframe):\n    os.makedirs(processed_folder, exist_ok=True)\n    for idx, row in dataframe.iterrows():\n        img_path = image_folder_1 + row[\'image_id\'] + \'.jpg\' if os.path.exists(image_folder_1 + row[\'image_id\'] + \'.jpg\') else image_folder_2 + row[\'image_id\'] + \'.jpg\'\n        masks = segment_image(img_path)\n        image = cv2.imread(img_path)\n        for i, mask in enumerate(masks):\n            mask_uint8 = (mask[\'segmentation\'] * 255).astype(np.uint8)\n            segmented_image = cv2.bitwise_and(image, image, mask=mask_uint8)\n            processed_path = os.path.join(processed_folder, f"{row[\'image_id\']}_mask_{i}.jpg")\n            cv2.imwrite(processed_path, segmented_image)\n\n        dataframe.at[idx, \'image_path\'] = processed_path\n\n\npreprocess_and_save_images(df)'

In [71]:
class SkinLesionDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_path = self.dataframe.iloc[idx]['image_path']
        label = self.dataframe.iloc[idx]['label_diagnosis']
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        return image, label

transform = transforms.Compose([transforms.Resize((224, 224)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])

In [72]:
train_dataset = SkinLesionDataset(train_df, transform=transform)
val_dataset = SkinLesionDataset(val_df, transform=transform)
test_dataset = SkinLesionDataset(test_df, transform=transform)

In [73]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [74]:
'''sam_checkpoint = '/content/drive/MyDrive/Курсовая/sam_vit_h_4b8939.pth'
model_type = 'vit_h'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint).to(device)
mask_generator = SamAutomaticMaskGenerator(sam)'''

"sam_checkpoint = '/content/drive/MyDrive/Курсовая/sam_vit_h_4b8939.pth'\nmodel_type = 'vit_h'\ndevice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')\nsam = sam_model_registry[model_type](checkpoint=sam_checkpoint).to(device)\nmask_generator = SamAutomaticMaskGenerator(sam)"

In [75]:
device

device(type='cuda')

In [76]:
model_name = 'google/vit-base-patch16-224-in21k'
model = ViTForImageClassification.from_pretrained(model_name, num_labels=len(class_names)).to(device) # предобученный vit
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

def train_model(num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct, total = 0, 0
        for batch_idx, (images, labels) in enumerate(train_dataloader):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            if batch_idx % 10 == 0:
                print(f'Эпоха {epoch+1}, Батч {batch_idx}/{len(train_dataloader)}, Потери: {running_loss/(batch_idx+1):.4f}')
        print(f'Эпоха {epoch+1}, Средние потери: {running_loss/len(train_dataloader):.4f}, Точность: {100 * correct / total:.2f}%')

train_model(num_epochs=10)

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Эпоха 1, Батч 0/220, Потери: 1.7831
Эпоха 1, Батч 10/220, Потери: 1.3658
Эпоха 1, Батч 20/220, Потери: 1.2602
Эпоха 1, Батч 30/220, Потери: 1.1237
Эпоха 1, Батч 40/220, Потери: 1.0540
Эпоха 1, Батч 50/220, Потери: 1.0181
Эпоха 1, Батч 60/220, Потери: 0.9835
Эпоха 1, Батч 70/220, Потери: 0.9509
Эпоха 1, Батч 80/220, Потери: 0.9236
Эпоха 1, Батч 90/220, Потери: 0.9073
Эпоха 1, Батч 100/220, Потери: 0.8870
Эпоха 1, Батч 110/220, Потери: 0.8675
Эпоха 1, Батч 120/220, Потери: 0.8545
Эпоха 1, Батч 130/220, Потери: 0.8391
Эпоха 1, Батч 140/220, Потери: 0.8294
Эпоха 1, Батч 150/220, Потери: 0.8129
Эпоха 1, Батч 160/220, Потери: 0.7993
Эпоха 1, Батч 170/220, Потери: 0.7907
Эпоха 1, Батч 180/220, Потери: 0.7806
Эпоха 1, Батч 190/220, Потери: 0.7716
Эпоха 1, Батч 200/220, Потери: 0.7640
Эпоха 1, Батч 210/220, Потери: 0.7561
Эпоха 1, Средние потери: 0.7492, Точность: 75.26%
Эпоха 2, Батч 0/220, Потери: 0.5932
Эпоха 2, Батч 10/220, Потери: 0.5206
Эпоха 2, Батч 20/220, Потери: 0.4959
Эпоха 2, Батч 3

In [77]:
def evaluate_model():
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Точность на тестовой выборке: {100 * correct / total:.2f}%')

In [78]:
evaluate_model()

Точность на тестовой выборке: 84.30%


In [80]:
model_save_path = '/content/drive/MyDrive/Курсовая/skin_lesion_model_25.01.25.pth'
torch.save(model.state_dict(), model_save_path)

Надо дорабатывать, она переобучена